In [1]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using LinearAlgebra
using Flux, DiffEqFlux
using PlotlyJS
using DifferentialEquations: DP5, Tsit5, Vern7, Vern9, BS3
using Random
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

WebIO._IJuliaInit()

false

In [2]:
ω₀ = 2π*5.0
η = -2π*300*1e-3
ωlo = ω₀

31.41592653589793

In [3]:
n_levels = 12
bs = FockBasis(n_levels-1)
a = destroy(bs)
ad = create(bs)
id = identityoperator(bs, bs);

In [4]:
H0 = ω₀*(ad*a + 0.5*id) + (η/12.0)*(a + ad)^4 - η^2 * (a + ad)^6/ω₀/90.0
H1 = 1.0im*(a - ad);

In [5]:
n_neurons = 8
Random.seed!(1)
ann = FastChain(FastDense(1, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh),
                FastDense(n_neurons, 2))
t0, t1 = 0.0, 4.0

I_guess(t) = @. -2π*exp(-(t-0.5*t1)^2/(0.2*t1)^2)*2.0*(t-0.5*t1) /(0.2*t1)^2
Q_guess(t) = @. -2π*exp(-(t-0.5*t1)^2/(0.2*t1)^2)

tsf32 = Float32(t0):0.001f0:Float32(t1)
Is = Vector{Float32}(I_guess(tsf32))
Qs = Vector{Float32}(Q_guess(tsf32))

ts = Vector{Float64}(tsf32)
function loss(p)
    c = 0.0f0
    for (i,t) in enumerate(tsf32)
        x = ann([t], p)
        c += (x[1] - Is[i])^2
        c += (x[2] - Qs[i])^2
    end
    c
end
res = DiffEqFlux.sciml_train(loss, initial_params(ann), ADAM(0.1f0), maxiters = 500)
θ = Vector{Float64}(res.u);
#θ = Vector{Float64}(initial_params(ann));

In [6]:
coeffs(params, t) = let b = ann([t], params)
                        [b[1]*cos(ωlo*t) + b[2]*sin(ωlo*t)]
                    end
bcs(params) = 0.01*sum(ann([t0], params).^2 + ann([t1], params).^2)
                 
cost = CostFunction((x,y)-> 1.0-real(x'*y), bcs)

CostFunction(var"#1#2"(), bcs)

In [7]:
trans = UnitaryTransform([fockstate(bs, 0), fockstate(bs, 1)], 
                         [[1.0 -1.0im];[-1.0im 1.0]]/√2);

In [8]:
tspan = (t0, t1)
H = Hamiltonian(H0, [H1], coeffs)
prob = QOCProblem(H, trans, tspan, cost);

In [9]:
@time sol = solve(prob, θ, ADAM(0.02); maxiter=200)

Progress: 100%|█████████████████████████████████████████| Time: 0:04:23
  distance:    0.004051665209568678
  contraints:  2.303269822766253e-5


278.588450 seconds (1.21 G allocations: 116.153 GiB, 26.66% gc time, 14.32% compilation time)


Solution{Float64}([-1.1306986611306162, -0.8425831245811041, -0.623546391936392, -0.057293001635333116, -1.2765358851803152, 0.777734717648167, -0.792724247101401, 0.4818008475057589, 0.9160872652599155, 1.424089949015126  …  4.422041178918318, 1.0892082336876976, -1.1083449682726871, 0.12624303517738575, 3.1803943253653464, 0.50464902211469, -1.2606743304546766, 1.7335362727380126, -0.01972996936904131, -0.14262863999400807], [1.1748429261214117, 0.9288213641733225, 0.5708557259503002, 0.2827521701427674, 0.13999372294690765, 0.09114892905631444, 0.07911441026714866, 0.08604308513912307, 0.10809634494604681, 0.1381976596236002  …  0.004088433471982733, 0.004084130869983882, 0.004079878840889062, 0.004075682039990147, 0.004071546118208869, 0.004067473137183475, 0.004063458131355002, 0.0040594917963445165, 0.0040555627256546045, 0.004051665209568678])

In [10]:
plot(sol.trace)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [11]:
I_wf(t) = ann([t], sol.params)[1]/2π
Q_wf(t) = ann([t], sol.params)[2]/2π

Q_wf (generic function with 1 method)

In [12]:
f= plot([
    scatter(x=ts, y=I_wf.(ts), name="I")
    scatter(x=ts, y=Q_wf.(ts), name="Q")
    ],
    Layout(
        xaxis_title_text="Time (ns)",
        yaxis_title_text="Frequency (GHz)",
        legend=attr(x=0, y=1,),
        font=attr(
            size=16,
        )
    )
)

savefig(f, "rxpi2_wfs.eps")

"rxpi2_wfs.eps"

In [13]:
tout, psit = schroedinger_dynamic(ts, fockstate(bs, 0), H, sol.params);

In [14]:
f = plot([
        scatter(x=ts, y=real(expect(dm(fockstate(bs, i-1)), psit)), name=string(i-1)) for i in 1:12 
    ],
    Layout(
        xaxis_title_text="Time (ns)",
        yaxis_title_text="Population (⟨i|ψ⟩²)",
        legend=attr(x=0, y=0.5,),
        font=attr(
            size=16,
        )
    )
)
savefig(f,"rxpi2_probs.eps")

"rxpi2_probs.eps"